# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [38]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

data_path = '../data/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [39]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
df['Cabin'] = df['Cabin'].fillna('None')
df['Age'] = df['Age'].fillna(df['Age'].mean())

mean_df = df.groupby(['Cabin'])['Age'].mean().reset_index()
mode_df = df.groupby(['Cabin'])['Age'].apply(lambda x: x.mode()[0]).reset_index()
median_df = df.groupby(['Cabin'])['Age'].median().reset_index()
max_df = df.groupby(['Cabin'])['Age'].max().reset_index()
temp = pd.merge(mean_df, mode_df, how='left', on=['Cabin'])
temp = pd.merge(temp, median_df, how='left', on=['Cabin'])
temp = pd.merge(temp, max_df, how='left', on=['Cabin'])
temp.columns = ['Cabin', 'Cabin_Age_Mean', 'Cabin_Age_Mode', 'Cabin_Age_Median', 'Cabin_Age_Max']
temp

,Cabin,Cabin_Pclass_Mean,Cabin_Pclass_Mode,Cabin_Pclass_Median,Cabin_Pclass_Max
0,A10,36.000000,36.000000,36.000000,36.000000
1,A14,29.699118,29.699118,29.699118,29.699118
2,A16,48.000000,48.000000,48.000000,48.000000
3,A19,29.699118,29.699118,29.699118,29.699118
4,A20,49.000000,49.000000,49.000000,49.000000
5,A23,80.000000,80.000000,80.000000,80.000000
6,A24,31.000000,31.000000,31.000000,31.000000
7,A26,56.000000,56.000000,56.000000,56.000000
8,A31,40.000000,40.000000,40.000000,40.000000
9,A32,29.699118,29.699118,29.699118,29.699118


In [40]:
df = pd.merge(df, temp, how='left', on=['Cabin'])
df = df.drop(['Cabin'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Cabin_Pclass_Mean,Cabin_Pclass_Mode,Cabin_Pclass_Median,Cabin_Pclass_Max
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S,28.048341,29.699118,29.699118,74.0
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C,38.000000,38.000000,38.000000,38.0
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S,28.048341,29.699118,29.699118,74.0
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S,36.000000,35.000000,36.000000,37.0
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S,28.048341,29.699118,29.699118,74.0


In [41]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin_Pclass_Mean', 'Cabin_Pclass_Mode', 'Cabin_Pclass_Median', 'Cabin_Pclass_Max']



,Pclass,Age,SibSp,Parch,Fare,Cabin_Pclass_Mean,Cabin_Pclass_Mode,Cabin_Pclass_Median,Cabin_Pclass_Max
0,3,22.0,1,0,7.2500,28.048341,29.699118,29.699118,74.0
1,1,38.0,1,0,71.2833,38.000000,38.000000,38.000000,38.0
2,3,26.0,0,0,7.9250,28.048341,29.699118,29.699118,74.0
3,1,35.0,1,0,53.1000,36.000000,35.000000,36.000000,37.0
4,3,35.0,0,0,8.0500,28.048341,29.699118,29.699118,74.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [42]:
df_minus = df.drop(['Cabin_Age_Mean', 'Cabin_Age_Mode', 'Cabin_Age_Median', 'Cabin_Age_Max'] , axis=1)


In [43]:
# 原始特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df_minus)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6982266036406296

In [44]:
# 新特徵 + 邏輯斯迴歸
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.708364497034705